In [1]:
import pandas as pd
import numpy as np
import re
import math
import tabula
import camelot
import pdfminer
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
try:
    from cStringIO import StringIO
except ImportError:
    from io import StringIO 
import matplotlib.pyplot as plt

In [3]:
f = r'C:\Users\rites\Desktop\pdf_to_excel-master\Axis_new\files\axis03.pdf'
tables=camelot.read_pdf(f,flavor='stream' ,pages = "all",edge_tol=500) 

In [71]:
df = pd.DataFrame; tmp = pd.DataFrame;

In [72]:
tables

<TableList n=6>

In [82]:
tmp = tables[3].df
tmp

,0,1,2,3,4,5,6,7
0,,,,,,,,CENTRALISED CTS MUM
1,20-11-2018,20-11-2018,NACH-DR- NEOGROWTH CREDIT PVT,,55834.00,DR,399813.23,MH
2,,,,,,,,RETAIL ASSET
3,20-11-2018,20-11-2018,BPR086102781710_EMI_20-11-2018,,36152.00,DR,363661.23,MANAGEMENT GR-RAM
4,20-11-2018,20-11-2018,NEFT/N324180169696088/LENDINGKART,,576.90,CR,364238.13,RTGS HUB
...,...,...,...,...,...,...,...,...
66,04-12-2018,04-12-2018,IMPS/P2A/833822353058/910020037211732/041218,,33505.90,CR,1197823.53,"POWAI, MUMBAI [MH]"
67,04-12-2018,04-12-2018,IMPS/P2A/833823365648/007863700000481YESB0000078,,33205.90,DR,1164617.63,"POWAI, MUMBAI [MH]"
68,,,,,,,,CENTRALISED CTS MUM
69,05-12-2018,05-12-2018,NACH-DR- EDELWEISSRETAILFINLT,,54981.00,DR,1109636.63,MH


In [5]:
tmp = tables[].df
try:
    idx=[ c for c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index ][0]
    tmp.columns=tmp.iloc[idx]
except:
    try:
        idx=[ c for c in dt[dt.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in dt[dt.apply(lambda row: row.astype(str).str.lower().str.contains('particular').any(), axis=1) ==True].index ][0]
        tmp.columns=tmp.iloc[idx]
    except:
        print("\nAxis-Column headers missing")
# tmp.shape[1]
# tmp=pd.DataFrame(tmp.T.drop_duplicates().T).replace(r'^\s*$', np.nan, regex=True)

In [6]:
tmp.columns

Index(['Tran Date', 'Value Date', 'Transaction Particulars', 'Chq No',
       'Amount(INR)', 'DR/CR', 'Balance(INR)', 'Branch Name'],
      dtype='object', name=2)

In [7]:
if len(tmp.columns) == 8:
    print("true")

true


In [8]:
if (tmp.shape[1] == 6):
    tmp[6] = np.nan
    tmp = tmp[[0,6,1,2,3,4,5]]
    tmp.columns = range(tmp.shape[1])

In [9]:
tmp.head()

2,Tran Date,Value Date,Transaction Particulars,Chq No,Amount(INR),DR/CR,Balance(INR),Branch Name
0,PIN400076,,,,,,,Currency :INR
1,,,,Statement of Axis Account No :910020037211732 ...,,,,
2,Tran Date,Value Date,Transaction Particulars,Chq No,Amount(INR),DR/CR,Balance(INR),Branch Name
3,,,OPENING BALANCE,,,,309677.68,
4,01-11-2017,01-11-2017,NEFT/171101076GN00206/FLIPKART INTERNET P LTD ...,,1158.38,CR,310836.06,RTGS HUB


In [152]:
def came(f):
    tables = camelot.read_pdf(f, flavor='stream', pages='1,2', edge_til=500)
    if len(tables) != 0:
        dt=pd.DataFrame(); tmp=pd.DataFrame()
        first_check = tables[0].df; second_check = tables[1].df;
        print(first_check.shape[1])
        print(second_check.shape[1])
        if (first_check.shape[1] > 5):
            try:
                idx=[ c for c in first_check[first_check.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in first_check[first_check.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index ][0]
                first_check.columns=first_check.iloc[idx]
            except:
                try:
                    idx=[ c for c in first_check[first_check.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in dt[dt.apply(lambda row: row.astype(str).str.lower().str.contains('particular').any(), axis=1) ==True].index ][0]
                    first_check.columns=first_check.iloc[idx]
                except:
                    print("\nAxis-Column headers missing")
            tables2 = camelot.read_pdf(f, flavor='stream', pages='all', edge_til=500)
            print("\nlength of first_check:",len(first_check.columns))
            if len(first_check.columns) == 7:
                print("Column length is 7")
                for i in range(len(tables2)):
                    tmp=tables2[i].df
                    if (tmp.shape[1] < 5):
                        print("\nDiiferent structure on page:",i); pass
                    elif (tmp.shape[1] == 7):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        dt = pd.concat([dt,tmp]).drop_duplicates(keep='first').reset_index(drop=True)
                    elif (tmp.shape[1] == 6):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        tmp[6] = np.nan; tmp = tmp[[0,6,1,2,3,4,5]]; tmp.columns = range(tmp.shape[1]);
                        dt = pd.concat([dt,tmp]).drop_duplicates(keep='first').reset_index(drop=True)
                    else:
                        print("\nAnother Structure on page:",i); pass

            elif len(first_check.columns) == 8:
                print("Column length is 8")
                for i in range(len(tables2)):
                    tmp=tables2[i].df   
                    if (tmp.shape[1] < 5):
                        print("\nDiiferent structure on page:",i); pass
                    elif (tmp.shape[1] == 8):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        dt = pd.concat([dt,tmp]).drop_duplicates(keep='first').reset_index(drop=True)
                    elif (tmp.shape[1] == 7):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        tmp[7] = np.nan; tmp = tmp[[0,1,2,7,3,4,5,6]]; tmp.columns = range(tmp.shape[1]);
                        dt = pd.concat([dt,tmp]).drop_duplicates(keep='first').reset_index(drop=True)
                    else:
                        print("\nAnother Structure on page:",i); pass
            elif len(first_check.columns) > 8:
                print("Column length greater than 8")
                for i in range(len(tables2)):
                    print(tmp.shape[1])
                    if (tmp.shape[1] > 8):
                        try:
                            idx=[ c for c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index ][0]
                            tmp.columns=tmp.iloc[idx] ; tmp=tmp.iloc[idx:,:] ; tmp.reset_index(drop=True,inplace=True)
                        except:
                            try:
                                idx=[ c for c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('particular').any(), axis=1) ==True].index ][0]
                                tmp.columns=tmp.iloc[idx] ; tmp=tmp.iloc[idx:,:] ; tmp.reset_index(drop=True,inplace=True)           
                            except:
                                print("\nAxis-Column headers missing"); pass
                        tmp.drop("",axis = 1,inplace = True)
                        tmp.columns = [0,1,2,3,4,5,6,7]
                        dt=pd.concat([dt,tmp]).drop_duplicates().reset_index(drop=True)
                    elif (tmp.shape[1] == 8):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        dt = pd.concat([dt,tmp]).drop_duplicates(keep='first').reset_index(drop=True)
                    elif (tmp.shape[1] == 7):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        tmp[7] = np.nan; tmp = tmp[[0,1,2,7,3,4,5,6]]; tmp.columns = range(tmp.shape[1]);
                        dt = pd.concat([dt,tmp]).drop_duplicates(keep='first').reset_index(drop=True)
                    elif (tmp.shape[1] == 6):
                        try:
                            idx=[ c for c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index ][0]
                        except:
                            try:
                                idx=[ c for c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('particular').any(), axis=1) ==True].index ][0]
                            except:
                                print("\nAxis-Column headers missing")
                        tmp.columns=tmp.iloc[idx] ; tmp=tmp.iloc[idx+1:,:] ; tmp.reset_index(drop=True,inplace=True) 
                        tmp=tmp[~tmp.index.isin(tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('page:|account status|total|reason for return|inward clg|opening balance|statement of a/c').any(), axis=1) ==True].index)]
                        if tmp.columns[0] == 'Tran Date\nValue Date\nTransaction Particulars':
                            tmp.columns = ["A","B","C","D","E","F"]
                            tmp.reset_index(drop = True, inplace=True)
                            tmp1 = pd.DataFrame(tmp.A.str.split('\n',2).tolist(),columns=["Z1", "Z2", "Z3"])
                            result = pd.concat([tmp1, tmp], axis=1, sort=False)
                            result.drop('A' ,axis=1, inplace=True)
                            result["Y"] = result["Z1"]
                            for i,j in enumerate(result["Y"]):
                                if re.search(r'(\d+-\d+-\d+)',j):
                                    result.at[i, 'Y'] = ""
                                    pass
                                elif re.search(r'^\s*$',j):
                                    result.at[i, 'Y'] = ""
                                    pass
                                else:
                                    pass
                                    
                            for i,j in enumerate(result["Z1"]):
                                if re.search(r'(\d+-\d+-\d+)',j):
                                    pass
                                elif re.search(r'^\s*$',j):
                                    result.at[i, 'Z1'] = ""
                                    pass
                                else:
                                    result.at[i, 'Z1'] = ""
                                    pass 
                            result.fillna(value=pd.np.nan, inplace=True)
                            result['X'] = result['Z3'].fillna('') + result['Y'].fillna('')
                            result.drop(['Z3','Y'] ,axis=1, inplace=True)
                            result = result[['Z1', 'Z2', 'X', 'B', 'C','D','E','F']]
                            result.columns = [0,1,2,3,4,5,6,7]
                            dt=pd.concat([dt,result]).drop_duplicates().reset_index(drop=True)
                        else:
                            print("\nIn without box pdf another Structure on page:",i); pass

        elif(second_check.shape[1] > 5):
            try:
                idx=[ c for c in second_check[second_check.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in second_check[second_check.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index ][0]
                second_check.columns=second_check.iloc[idx]
            except:
                try:
                    idx=[ c for c in second_check[second_check.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in dt[dt.apply(lambda row: row.astype(str).str.lower().str.contains('particular').any(), axis=1) ==True].index ][0]
                    second_check.columns=second_check.iloc[idx]
                except:
                    print("\nAxis-Column headers missing")
            tables2 = camelot.read_pdf(f, flavor='stream', pages='all', edge_til=500)
            if len(second_check.columns) == 7:
                print("Column length is 7")
                for i in range(len(tables2)):
                    tmp=tables2[i].df
                    if (tmp.shape[1] < 5):
                        print("\nDiiferent structure on page:",i); pass
                    elif (tmp.shape[1] == 7):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        dt = pd.concat([dt,tmp]).drop_duplicates(keep='first').reset_index(drop=True)
                    elif (tmp.shape[1] == 6):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        tmp[6] = np.nan; tmp = tmp[[0,6,1,2,3,4,5]]; tmp.columns = range(tmp.shape[1]);
                        dt = pd.concat([dt,tmp]).drop_duplicates(keep='first').reset_index(drop=True)
                    else:
                        print("\nAnother Structure on page:",i); pass

            elif len(second_check.columns) == 8:
                print("Column length is 8")
                for i in range(len(tables2)):
                    print("\n",i,":",tmp.shape[1])
                    tmp=tables2[i].df
                    if (tmp.shape[1] < 5):
                        print("\nDiiferent structure on page:",i); pass
                    elif (tmp.shape[1] == 8):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        dt = pd.concat([dt,tmp]).drop_duplicates(keep='first').reset_index(drop=True)
                    elif (tmp.shape[1] == 7):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        tmp[7] = np.nan; tmp = tmp[[0,1,2,7,3,4,5,6]]; tmp.columns = range(tmp.shape[1]);
                        dt = pd.concat([dt,tmp]).drop_duplicates(keep='first').reset_index(drop=True)
                    else:
                        print("\nAnother Structure on page:",i); pass

    else:
        print("\nLength of table is 0"); pass   
    
    return dt

f = r'C:\Users\MudraCircle\Desktop\bks_raw\Parsing_testing\Axis\files\axis02.pdf'
output = came(f)
output.to_csv("output.csv", index=0)
output

8
7

length of first_check: 8
Column length is 8

Diiferent structure on page: 30


,0,1,2,3,4,5,6,7
0,PIN400076,NaN,NaN,NaN,NaN,NaN,NaN,Currency :INR
1,NaN,NaN,NaN,Statement of Axis Account No :910020037211732 ...,NaN,NaN,NaN,NaN
2,Tran Date,Value Date,Transaction Particulars,Chq No,Amount(INR),DR/CR,Balance(INR),Branch Name
3,NaN,NaN,OPENING BALANCE,NaN,NaN,NaN,309677.68,NaN
4,01-11-2017,01-11-2017,NEFT/171101076GN00206/FLIPKART INTERNET P LTD ...,NaN,1158.38,CR,310836.06,RTGS HUB
...,...,...,...,...,...,...,...,...
1673,31-10-2018,31-10-2018,IMPS/P2A/830412412196/919768667157/ATOMNETBANKIN,NaN,145000.00,CR,555297.47,"POWAI, MUMBAI [MH]"
1674,31-10-2018,31-10-2018,NEFT/N304180163207887/LENDINGKART,NaN,6730.70,CR,562028.17,DATA CENTRE-CHEMBUR
1675,31-10-2018,31-10-2018,NEFT/N304180163208334/LENDINGKART,NaN,178.89,CR,562207.06,RTGS HUB
1676,NaN,NaN,TRANSACTION TOTAL DR/CR,NaN,47314531.11/47567060.49,NaN,NaN,NaN


# ******************************

In [5]:
def came(f):
    tables = camelot.read_pdf(f, flavor='stream', pages='1,2', edge_til=500)
    if len(tables) != 0:
        dt=pd.DataFrame(); tmp=pd.DataFrame()
        first_check = tables[0].df; second_check = tables[1].df;
        print(first_check.shape[1])
        print(second_check.shape[1])
        if (first_check.shape[1] > 5):
            try:
                idx=[ c for c in first_check[first_check.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in first_check[first_check.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index ][0]
                first_check.columns=first_check.iloc[idx]
            except:
                try:
                    idx=[ c for c in first_check[first_check.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in dt[dt.apply(lambda row: row.astype(str).str.lower().str.contains('particular').any(), axis=1) ==True].index ][0]
                    first_check.columns=first_check.iloc[idx]
                except:
                    print("\nAxis-Column headers missing")
            tables2 = camelot.read_pdf(f, flavor='stream', pages='all', edge_til=500)
            print("\nlength of first_check:",len(first_check.columns))
            if len(first_check.columns) == 7:
                print("Column length is 7")
                for i in range(len(tables2)):
                    tmp=tables2[i].df
                    if (tmp.shape[1] < 5):
                        print("\nDiiferent structure on page:",i); pass
                    elif (tmp.shape[1] == 7):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        dt = pd.concat([dt,tmp]).drop_duplicates(keep='first').reset_index(drop=True)
                    elif (tmp.shape[1] == 6):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        tmp[6] = np.nan; tmp = tmp[[0,6,1,2,3,4,5]]; tmp.columns = range(tmp.shape[1]);
                        dt = pd.concat([dt,tmp]).drop_duplicates(keep='first').reset_index(drop=True)
                    else:
                        print("\nAnother Structure on page:",i); pass

            elif len(first_check.columns) == 8:
                print("Column length is 8")
                for i in range(len(tables2)):
                    tmp=tables2[i].df   
                    if (tmp.shape[1] < 5):
                        print("\nDiiferent structure on page:",i); pass
                    elif (tmp.shape[1] == 8):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        dt = pd.concat([dt,tmp]).drop_duplicates(keep='first').reset_index(drop=True)
                    elif (tmp.shape[1] == 7):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        tmp[7] = np.nan; tmp = tmp[[0,1,2,7,3,4,5,6]]; tmp.columns = range(tmp.shape[1]);
                        dt = pd.concat([dt,tmp]).drop_duplicates(keep='first').reset_index(drop=True)
                    else:
                        print("\nAnother Structure on page:",i); pass
            elif len(first_check.columns) > 8:
                print("Column length greater than 8")
                for i in range(len(tables2)):
                    tmp=tables2[i].df 
                    print("////////////////////////////////////////////////////////////////////////////////////////////")
                    print("\n",tmp.shape[1])
                    print("////////////////////////////////////////////////////////////////////////////////////////////")
                    if (tmp.shape[1] > 8):
                        try:
                            idx=[ c for c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index ][0]
                            tmp.columns=tmp.iloc[idx] ; tmp=tmp.iloc[idx:,:] ; tmp.reset_index(drop=True,inplace=True)
                        except:
                            try:
                                idx=[ c for c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('particular').any(), axis=1) ==True].index ][0]
                                tmp.columns=tmp.iloc[idx] ; tmp=tmp.iloc[idx:,:] ; tmp.reset_index(drop=True,inplace=True)           
                            except:
                                print("\nAxis-Column headers missing"); pass
                        tmp.drop("",axis = 1,inplace = True)
                        tmp.columns = [0,1,2,3,4,5,6,7]
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        print("\n",tmp)
                        dt=pd.concat([dt,tmp]).drop_duplicates().reset_index(drop=True)
                    elif (tmp.shape[1] == 8):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        print("\n",tmp)
                        dt = pd.concat([dt,tmp]).drop_duplicates(keep='first').reset_index(drop=True)
                    elif (tmp.shape[1] == 7):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        tmp[7] = np.nan; tmp = tmp[[0,1,2,7,3,4,5,6]]; tmp.columns = range(tmp.shape[1]);
                        print("\n",tmp)
                        dt = pd.concat([dt,tmp]).drop_duplicates(keep='first').reset_index(drop=True)
                    elif (tmp.shape[1] == 6):
                        try:
                            idx=[ c for c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index ][0]
                        except:
                            try:
                                idx=[ c for c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('particular').any(), axis=1) ==True].index ][0]
                            except:
                                print("\nAxis-Column headers missing")
                        tmp.columns=tmp.iloc[idx] ; tmp=tmp.iloc[idx+1:,:] ; tmp.reset_index(drop=True,inplace=True) 
                        tmp=tmp[~tmp.index.isin(tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('page:|account status|total|reason for return|inward clg|opening balance|statement of a/c').any(), axis=1) ==True].index)]
                        if tmp.columns[0] == 'Tran Date\nValue Date\nTransaction Particulars':
                            tmp.columns = ["A","B","C","D","E","F"]
                            tmp.reset_index(drop = True, inplace=True)
                            tmp1 = pd.DataFrame(tmp.A.str.split('\n',2).tolist(),columns=["Z1", "Z2", "Z3"])
                            result = pd.concat([tmp1, tmp], axis=1, sort=False)
                            result.drop('A' ,axis=1, inplace=True)
                            result["Y"] = result["Z1"]
                            for i,j in enumerate(result["Y"]):
                                if re.search(r'(\d+-\d+-\d+)',j):
                                    result.at[i, 'Y'] = ""
                                    pass
                                elif re.search(r'^\s*$',j):
                                    result.at[i, 'Y'] = ""
                                    pass
                                else:
                                    pass
                                    
                            for i,j in enumerate(result["Z1"]):
                                if re.search(r'(\d+-\d+-\d+)',j):
                                    pass
                                elif re.search(r'^\s*$',j):
                                    result.at[i, 'Z1'] = ""
                                    pass
                                else:
                                    result.at[i, 'Z1'] = ""
                                    pass 
                            result.fillna(value=pd.np.nan, inplace=True)
                            result['X'] = result['Z3'].fillna('') + result['Y'].fillna('')
                            result.drop(['Z3','Y'] ,axis=1, inplace=True)
                            result = result[['Z1', 'Z2', 'X', 'B', 'C','D','E','F']]
                            result.columns = [0,1,2,3,4,5,6,7]
                            result=pd.DataFrame(result).replace(r'^\s*$', np.nan, regex=True)
                            print("\n",result)
                            dt=pd.concat([dt,result]).drop_duplicates().reset_index(drop=True)
                        else:
                            print("\nIn without box pdf another Structure on page:",i); pass

        elif(second_check.shape[1] > 5):
            try:
                idx=[ c for c in second_check[second_check.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in second_check[second_check.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index ][0]
                second_check.columns=second_check.iloc[idx]
            except:
                try:
                    idx=[ c for c in second_check[second_check.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in dt[dt.apply(lambda row: row.astype(str).str.lower().str.contains('particular').any(), axis=1) ==True].index ][0]
                    second_check.columns=second_check.iloc[idx]
                except:
                    print("\nAxis-Column headers missing")
            tables2 = camelot.read_pdf(f, flavor='stream', pages='all', edge_til=500)
            if len(second_check.columns) == 7:
                print("Column length is 7")
                for i in range(len(tables2)):
                    tmp=tables2[i].df
                    if (tmp.shape[1] < 5):
                        print("\nDiiferent structure on page:",i); pass
                    elif (tmp.shape[1] == 7):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        dt = pd.concat([dt,tmp]).drop_duplicates(keep='first').reset_index(drop=True)
                    elif (tmp.shape[1] == 6):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        tmp[6] = np.nan; tmp = tmp[[0,6,1,2,3,4,5]]; tmp.columns = range(tmp.shape[1]);
                        dt = pd.concat([dt,tmp]).drop_duplicates(keep='first').reset_index(drop=True)
                    else:
                        print("\nAnother Structure on page:",i); pass

            elif len(second_check.columns) == 8:
                print("Column length is 8")
                for i in range(len(tables2)):
                    print("\n",i,":",tmp.shape[1])
                    tmp=tables2[i].df
                    if (tmp.shape[1] < 5):
                        print("\nDiiferent structure on page:",i); pass
                    elif (tmp.shape[1] == 8):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        dt = pd.concat([dt,tmp]).drop_duplicates(keep='first').reset_index(drop=True)
                    elif (tmp.shape[1] == 7):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        tmp[7] = np.nan; tmp = tmp[[0,1,2,7,3,4,5,6]]; tmp.columns = range(tmp.shape[1]);
                        dt = pd.concat([dt,tmp]).drop_duplicates(keep='first').reset_index(drop=True)
                    else:
                        print("\nAnother Structure on page:",i); pass

    else:
        print("\nLength of table is 0"); pass   
    
    return dt

f = r'C:\Users\rites\Desktop\pdf_to_excel-master\Axis_new\files\axis03.pdf'
output = came(f)
output.to_csv("output02.csv", index=0)
output

9
8

length of first_check: 9
Column length greater than 8
////////////////////////////////////////////////////////////////////////////////////////////

 9
////////////////////////////////////////////////////////////////////////////////////////////

              0           1                                                 2  \
0    Tran Date  Value Date                           Transaction Particulars   
1                                                          OPENING BALANCE :   
2                                                                              
3   03-12-2018  03-12-2018                 EDC/00049-00051-/M037011032984700   
4                                                                              
5                                                                              
6   06-12-2018  06-12-2018             ECOM PUR/PTM*PAYTM/NOIDA/061218/09:18   
7                                                                              
8                            

c:\users\rites\appdata\local\programs\python\python36-32\lib\site-packages\pandas\core\frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,



              0           1                                                 2  \
0                      NaN                                                     
1   14-01-2019  14-01-2019                 EDC/00053-00053-/M037011032984700   
2                      NaN                                                     
3                      NaN                                                     
4   14-01-2019  14-01-2019  IMPS/P2A/901412924696/919971624523/INETIMPS00073   
5                      NaN                                                     
6                      NaN                                                     
7   14-01-2019  14-01-2019  IMPS/P2A/901412927199/919971624523/MOBUA01304940   
8                      NaN                                                     
9                      NaN                                                     
10  14-01-2019  14-01-2019   IMPS/P2A/901412321960/0831010104411UTBI0MBZC45/   
11                     NaN            

              0           1                                               2  \
0    Tran Date  Value Date                         Transaction Particulars   
1          NaN         NaN                               OPENING BALANCE :   
2          NaN         NaN                                             NaN   
3   10-08-2019  10-08-2019                    Consolidated Charges for A/c   
4          NaN         NaN                                             NaN   
5          NaN         NaN                                             NaN   
6   12-08-2019  12-08-2019           ECOM PUR/PTM*PAYTM/NOIDA/120819/12:03   
7          NaN         NaN                                             NaN   
8          NaN         NaN                                             NaN   
9   13-08-2019  13-08-2019           NEFT/MB/AXMB192258016621/RANJIT DUTTA   
10         NaN         NaN                                             NaN   
11         NaN         NaN                                     

,0,1,2,3,4,5,6,7
0,Tran Date,Value Date,Transaction Particulars,Chq No.,Amount,DR|CR,Balance,Branch Name
1,,,OPENING BALANCE :,,,,25184.55,
2,,,,,,,,ATM
3,03-12-2018,03-12-2018,EDC/00049-00051-/M037011032984700,,5053.95,CR,30238.50,RECONCILATION
4,,,,,,,,CENTRE
...,...,...,...,...,...,...,...,...
281,10-11-2019,10-11-2019,ECOM PUR/One97 Communi/Noida/101119/19:04,NaN,334.98,DR,4748.06,NaN
282,15-11-2019,15-11-2019,IMPS/P2A/931916091620/RANJIT/HDFCBAN/X080290/,NaN,202.95,DR,4545.11,NaN
283,16-11-2019,16-11-2019,NEFT/N320190983426385/INDU BHARGAVA/Payment,NaN,3300.00,CR,7845.11,RTGS HUB
284,20-11-2019,20-11-2019,ECOM PUR/One97 Communi/Noida/201119/15:01,NaN,60.00,DR,7785.11,NaN


# Perfect One

In [6]:
def came(f):
    tables = camelot.read_pdf(f, flavor='stream', pages='1,2', edge_til=500)
    if len(tables) != 0:
        dt=pd.DataFrame(); tmp=pd.DataFrame()
        first_check = tables[0].df; second_check = tables[1].df;
        print(first_check.shape[1])
        print(second_check.shape[1])
        if (first_check.shape[1] > 5):
            try:
                idx=[ c for c in first_check[first_check.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in first_check[first_check.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index ][0]
                first_check.columns=first_check.iloc[idx]
            except:
                try:
                    idx=[ c for c in first_check[first_check.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in dt[dt.apply(lambda row: row.astype(str).str.lower().str.contains('particular').any(), axis=1) ==True].index ][0]
                    first_check.columns=first_check.iloc[idx]
                except:
                    print("\nAxis-Column headers missing")
            tables2 = camelot.read_pdf(f, flavor='stream', pages='all', edge_til=500)
            print("\nlength of first_check:",len(first_check.columns))
            if len(first_check.columns) == 7:
                print("Column length is 7")
                for i in range(len(tables2)):
                    tmp=tables2[i].df
                    if (tmp.shape[1] < 5):
                        print("\nDiiferent structure on page:",i); pass
                    elif (tmp.shape[1] == 7):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        dt = pd.concat([dt,tmp]).drop_duplicates(keep='first').reset_index(drop=True)
                    elif (tmp.shape[1] == 6):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        tmp[6] = np.nan; tmp = tmp[[0,6,1,2,3,4,5]]; tmp.columns = range(tmp.shape[1]);
                        dt = pd.concat([dt,tmp]).drop_duplicates(keep='first').reset_index(drop=True)
                    else:
                        print("\nAnother Structure on page:",i); pass

            elif len(first_check.columns) == 8:
                print("Column length is 8")
                for i in range(len(tables2)):
                    tmp=tables2[i].df   
                    if (tmp.shape[1] < 5):
                        print("\nDiiferent structure on page:",i); pass
                    elif (tmp.shape[1] == 8):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        dt = pd.concat([dt,tmp]).drop_duplicates(keep='first').reset_index(drop=True)
                    elif (tmp.shape[1] == 7):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        tmp[7] = np.nan; tmp = tmp[[0,1,2,7,3,4,5,6]]; tmp.columns = range(tmp.shape[1]);
                        dt = pd.concat([dt,tmp]).drop_duplicates(keep='first').reset_index(drop=True)
                    else:
                        print("\nAnother Structure on page:",i); pass
            elif len(first_check.columns) > 8:
                print("Column length greater than 8")
                for i in range(len(tables2)):
                    tmp=tables2[i].df 
                    print("////////////////////////////////////////////////////////////////////////////////////////////")
                    print("\n",tmp.shape[1])
                    print("////////////////////////////////////////////////////////////////////////////////////////////")
                    if (tmp.shape[1] > 8):
                        try:
                            idx=[ c for c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index ][0]
                            tmp.columns=tmp.iloc[idx] ; tmp=tmp.iloc[idx:,:] ; tmp.reset_index(drop=True,inplace=True)
                        except:
                            try:
                                idx=[ c for c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('particular').any(), axis=1) ==True].index ][0]
                                tmp.columns=tmp.iloc[idx] ; tmp=tmp.iloc[idx:,:] ; tmp.reset_index(drop=True,inplace=True)           
                            except:
                                print("\nAxis-Column headers missing"); pass
                        tmp.drop("",axis = 1,inplace = True)
                        tmp.columns = [0,1,2,3,4,5,6,7]
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        print("\n",tmp)
                        dt=pd.concat([dt,tmp]).drop_duplicates().reset_index(drop=True)
                    elif (tmp.shape[1] == 8):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        print("\n",tmp)
                        dt = pd.concat([dt,tmp])
                    elif (tmp.shape[1] == 7):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        tmp[7] = np.nan; tmp = tmp[[0,1,2,7,3,4,5,6]]; tmp.columns = range(tmp.shape[1]);
                        print("\n",tmp)
                        dt = pd.concat([dt,tmp])
                    elif (tmp.shape[1] == 6):
                        try:
                            idx=[ c for c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index ][0]
                        except:
                            try:
                                idx=[ c for c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('particular').any(), axis=1) ==True].index ][0]
                            except:
                                print("\nAxis-Column headers missing")
                        tmp.columns=tmp.iloc[idx] ; tmp=tmp.iloc[idx+1:,:] ; tmp.reset_index(drop=True,inplace=True) 
                        tmp=tmp[~tmp.index.isin(tmp[tmp.apply(lambda row: row.astype(str).str.lower().str.contains('page:|account status|total|reason for return|inward clg|opening balance|statement of a/c').any(), axis=1) ==True].index)]
                        if tmp.columns[0] == 'Tran Date\nValue Date\nTransaction Particulars':
                            tmp.columns = ["A","B","C","D","E","F"]
                            tmp.reset_index(drop = True, inplace=True)
                            tmp1 = pd.DataFrame(tmp.A.str.split('\n',2).tolist(),columns=["Z1", "Z2", "Z3"])
                            result = pd.concat([tmp1, tmp], axis=1, sort=False)
                            result.drop('A' ,axis=1, inplace=True)
                            result["Y"] = result["Z1"]
                            for i,j in enumerate(result["Y"]):
                                if re.search(r'(\d+-\d+-\d+)',j):
                                    result.at[i, 'Y'] = ""
                                    pass
                                elif re.search(r'^\s*$',j):
                                    result.at[i, 'Y'] = ""
                                    pass
                                else:
                                    pass
                                    
                            for i,j in enumerate(result["Z1"]):
                                if re.search(r'(\d+-\d+-\d+)',j):
                                    pass
                                elif re.search(r'^\s*$',j):
                                    result.at[i, 'Z1'] = ""
                                    pass
                                else:
                                    result.at[i, 'Z1'] = ""
                                    pass 
                            result.fillna(value=pd.np.nan, inplace=True)
                            result['X'] = result['Z3'].fillna('') + result['Y'].fillna('')
                            result.drop(['Z3','Y'] ,axis=1, inplace=True)
                            result = result[['Z1', 'Z2', 'X', 'B', 'C','D','E','F']]
                            result.columns = [0,1,2,3,4,5,6,7]
                            result=pd.DataFrame(result).replace(r'^\s*$', np.nan, regex=True)
                            print("\n",result)
                            dt=pd.concat([dt,result])
                        else:
                            print("\nIn without box pdf another Structure on page:",i); pass

        elif(second_check.shape[1] > 5):
            try:
                idx=[ c for c in second_check[second_check.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in second_check[second_check.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index ][0]
                second_check.columns=second_check.iloc[idx]
            except:
                try:
                    idx=[ c for c in second_check[second_check.apply(lambda row: row.astype(str).str.lower().str.contains('balance').any(), axis=1) ==True].index if c in dt[dt.apply(lambda row: row.astype(str).str.lower().str.contains('particular').any(), axis=1) ==True].index ][0]
                    second_check.columns=second_check.iloc[idx]
                except:
                    print("\nAxis-Column headers missing")
            tables2 = camelot.read_pdf(f, flavor='stream', pages='all', edge_til=500)
            if len(second_check.columns) == 7:
                print("Column length is 7")
                for i in range(len(tables2)):
                    tmp=tables2[i].df
                    if (tmp.shape[1] < 5):
                        print("\nDiiferent structure on page:",i); pass
                    elif (tmp.shape[1] == 7):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        dt = pd.concat([dt,tmp]).drop_duplicates(keep='first').reset_index(drop=True)
                    elif (tmp.shape[1] == 6):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        tmp[6] = np.nan; tmp = tmp[[0,6,1,2,3,4,5]]; tmp.columns = range(tmp.shape[1]);
                        dt = pd.concat([dt,tmp]).drop_duplicates(keep='first').reset_index(drop=True)
                    else:
                        print("\nAnother Structure on page:",i); pass

            elif len(second_check.columns) == 8:
                print("Column length is 8")
                for i in range(len(tables2)):
                    print("\n",i,":",tmp.shape[1])
                    tmp=tables2[i].df
                    if (tmp.shape[1] < 5):
                        print("\nDiiferent structure on page:",i); pass
                    elif (tmp.shape[1] == 8):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        dt = pd.concat([dt,tmp]).drop_duplicates(keep='first').reset_index(drop=True)
                    elif (tmp.shape[1] == 7):
                        tmp=pd.DataFrame(tmp).replace(r'^\s*$', np.nan, regex=True)
                        tmp[7] = np.nan; tmp = tmp[[0,1,2,7,3,4,5,6]]; tmp.columns = range(tmp.shape[1]);
                        dt = pd.concat([dt,tmp]).drop_duplicates(keep='first').reset_index(drop=True)
                    else:
                        print("\nAnother Structure on page:",i); pass

    else:
        print("\nLength of table is 0"); pass   
    
    return dt

f = r'C:\Users\rites\Desktop\pdf_to_excel-master\Axis_new\files\axis03.pdf'
output = came(f)
output.to_csv("output03.csv", index=0)
output

9
8

length of first_check: 9
Column length greater than 8
////////////////////////////////////////////////////////////////////////////////////////////

 9
////////////////////////////////////////////////////////////////////////////////////////////

              0           1                                                 2  \
0    Tran Date  Value Date                           Transaction Particulars   
1          NaN         NaN                                 OPENING BALANCE :   
2          NaN         NaN                                               NaN   
3   03-12-2018  03-12-2018                 EDC/00049-00051-/M037011032984700   
4          NaN         NaN                                               NaN   
5          NaN         NaN                                               NaN   
6   06-12-2018  06-12-2018             ECOM PUR/PTM*PAYTM/NOIDA/061218/09:18   
7          NaN         NaN                                               NaN   
8          NaN         NaN   

c:\users\rites\appdata\local\programs\python\python36-32\lib\site-packages\pandas\core\frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,



              0           1                                                 2  \
0    Tran Date  Value Date                           Transaction Particulars   
1          NaN         NaN                                 OPENING BALANCE :   
2          NaN         NaN                                  BY CASH DEPOSIT-   
3   20-12-2018  20-12-2018                                               NaN   
4          NaN         NaN                BNA/DPRH329801/1538/201218/SOUTH D   
5          NaN         NaN                                               NaN   
6   20-12-2018  20-12-2018             NEFT/MB/AXMB183547203664/RANJIT DUTTA   
7          NaN         NaN                                               NaN   
8          NaN         NaN                                               NaN   
9   20-12-2018  20-12-2018                           By Clg 000277 485 DELHI   
10         NaN         NaN                                               NaN   
11         NaN         NaN            

44 NaN       NaN  NaN       NaN         DEL DL  
////////////////////////////////////////////////////////////////////////////////////////////

 8
////////////////////////////////////////////////////////////////////////////////////////////

              0           1                                                 2  \
0    Tran Date  Value Date                           Transaction Particulars   
1          NaN         NaN                                 OPENING BALANCE :   
2          NaN         NaN                                  BY CASH DEPOSIT-   
3   28-01-2019  28-01-2019                                               NaN   
4          NaN         NaN                BNA/DPRH329801/8915/280119/SOUTH D   
5          NaN         NaN                                               NaN   
6   28-01-2019  28-01-2019           MOB/SELFFT/RANJIT DUTTA/918010003842859   
7          NaN         NaN                                               NaN   
8          NaN         NaN             


              0           1                                              2  \
0    Tran Date  Value Date                        Transaction Particulars   
1          NaN         NaN                              OPENING BALANCE :   
2          NaN         NaN                                            NaN   
3   22-10-2019  22-10-2019          ECOM PUR/PTM*PAYTM/NOIDA/221019/13:47   
4          NaN         NaN                                            NaN   
5          NaN         NaN                                            NaN   
6   24-10-2019  24-10-2019        Dr Card Charges ANNUAL 5223XXXXXXXX6601   
7          NaN         NaN                                            NaN   
8          NaN         NaN                                            NaN   
9   24-10-2019  24-10-2019      ECOM PUR/One97 Communi/Noida/241019/20:51   
10         NaN         NaN                                            NaN   
11         NaN         NaN                                            NaN 

,0,1,2,3,4,5,6,7
0,Tran Date,Value Date,Transaction Particulars,Chq No.,Amount,DR|CR,Balance,Branch Name
1,NaN,NaN,OPENING BALANCE :,NaN,NaN,NaN,25184.55,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ATM
3,03-12-2018,03-12-2018,EDC/00049-00051-/M037011032984700,NaN,5053.95,CR,30238.50,RECONCILATION
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CENTRE
...,...,...,...,...,...,...,...,...
45,16-11-2019,16-11-2019,NEFT/N320190983426385/INDU BHARGAVA/Payment,NaN,3300.00,CR,7845.11,RTGS HUB
46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZAMROODPUR
47,20-11-2019,20-11-2019,ECOM PUR/One97 Communi/Noida/201119/15:01,NaN,60.00,DR,7785.11,NaN
48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DEL DL


In [147]:
f = r'C:\Users\MudraCircle\Desktop\bks_raw\Parsing_testing\Axis\files\axis02.pdf'
tables = camelot.read_pdf(f, flavor='stream', pages='all', edge_til=500)
if len(tables) != 0:
    df=pd.DataFrame(); tmp=pd.DataFrame();
    first_check = tables[0].df; second_check = tables[1].df;

    if (first_check.shape[1] < 5):
        print("first_check")

    elif(second_check.shape[1] > 5):
        print("second_check")

second_check


In [148]:
tables

<TableList n=31>

In [144]:
tables[8].df

,0,1,2,3,4,5,6
0,01-02-2018,01-02-2018,IMPS/P2A/803208798380/194602000000357IOBA0001946,200017.70,DR,2305171.63,"POWAI, MUMBAI [MH]"
1,01-02-2018,01-02-2018,IMPS/P2A/803208798705/194602000000197IOBA0001946,200017.70,DR,2105153.93,"POWAI, MUMBAI [MH]"
2,01-02-2018,01-02-2018,IMPS/P2A/803208888541/919768667157/to aaditya c,200000.00,CR,2305153.93,"POWAI, MUMBAI [MH]"
3,01-02-2018,01-02-2018,IMPS/P2A/803208800533/194602000000197IOBA0001946,165017.70,DR,2140136.23,"POWAI, MUMBAI [MH]"
4,01-02-2018,01-02-2018,IMPS/P2A/803208800774/194602000000357IOBA0001946,135017.70,DR,2005118.53,"POWAI, MUMBAI [MH]"
...,...,...,...,...,...,...,...
57,,,,,,,CENTRALISED CTS MUM
58,08-02-2018,08-02-2018,NACH-DR- INTELLECASH MICROFIN,102083.00,DR,1467036.01,MH
59,08-02-2018,08-02-2018,IMPS/P2A/803914490628/910000000000/aaditya,72000.00,CR,1539036.01,"POWAI, MUMBAI [MH]"
60,,,,,,,CENTRALISED COLL &
